In [1]:
import os
import numpy as np
from PIL import Image
from collections import Counter

def load_rgb_mask(path):
    """Load an RGB mask image and return it as a (H, W, 3) NumPy array."""
    return np.array(Image.open(path).convert('RGB'))

def majority_vote_rgb(masks_stack):
    """Perform majority voting on stacked RGB masks of shape (H, W, 3, N)."""
    H, W, _, N = masks_stack.shape
    # Reshape to (H * W, 3, N)
    flat_stack = masks_stack.reshape(-1, 3, N)

    # Transpose to (H*W, N, 3) for pixel-wise comparison
    flat_stack = np.transpose(flat_stack, (0, 2, 1))

    voted_pixels = []
    for pixels in flat_stack:
        rgb_tuples = [tuple(rgb) for rgb in pixels]
        most_common = Counter(rgb_tuples).most_common(1)[0][0]
        voted_pixels.append(most_common)

    # Reshape to (H, W, 3)
    return np.array(voted_pixels, dtype=np.uint8).reshape(H, W, 3)

def rgb_voting_postprocess(mask_folders, output_folder):
    """
    Args:
        mask_folders: List of folder paths containing RGB segmentation masks.
        output_folder: Path to save the voted masks.
    """
    os.makedirs(output_folder, exist_ok=True)

    # Get list of file names from the first folder
    file_names = sorted(os.listdir(mask_folders[0]))

    for file_name in file_names:
        masks = []
        for folder in mask_folders:
            mask_path = os.path.join(folder, file_name)
            masks.append(load_rgb_mask(mask_path))

        # Stack masks to shape (H, W, 3, N)
        masks_stack = np.stack(masks, axis=-1)  # shape (H, W, 3, N)

        # Apply pixel-wise majority voting
        voted_mask = majority_vote_rgb(masks_stack)

        # Save result
        Image.fromarray(voted_mask).save(os.path.join(output_folder, file_name))

# Example usage
mask_folders = [
    "6739",
    "6682",
    "6711"
]
output_folder = "voted_masks"
rgb_voting_postprocess(mask_folders, output_folder)
